In [1]:
import pandas as pd

# 1.a. Rules

In [7]:
input_path = "data/"
output_path = "output/grammar.txt"

In [62]:
df = pd.read_excel(f"{input_path}/sample_grammar_rules.xlsx")
df.head()

,NP -> ART N
0,NP -> ART ADJ N
1,NP -> ART N PP
2,NP -> ART ADJ ADJ N
3,NP -> DET N
4,NP -> PRON


In [63]:
import os
print("Thư mục hiện tại:", os.getcwd())

Thư mục hiện tại: d:\HCMUT\241\CO3085_NLP\assignment1


In [64]:
output_directory = 'output'
os.makedirs(output_directory, exist_ok=True)

output_path = os.path.join(output_directory, 'grammar.txt')

with open(output_path, 'w', encoding='utf-8') as f:
        for index, row in df.iterrows():
            line = ', '.join(str(value) for value in row) 
            f.write(line + '\n')  

In [67]:
def convert_grammar_format(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    converted_rules = {}
    for line in lines:
        line = line.strip()
        lhs, rhs = line.split('->')
        lhs = lhs.strip()
        rhs_parts = rhs.strip().split()

        if lhs in converted_rules:
            converted_rules[lhs].append(rhs_parts)
        else:
            converted_rules[lhs] = [rhs_parts]

    with open(output_file, 'w', encoding='utf-8') as f:
        for key, value in converted_rules.items():
            f.write(f"'{key}': {value},\n")

input_file = "output/grammar.txt"  # Tên file đầu vào
output_file = "output/output.txt"  # Tên file đầu ra
convert_grammar_format(input_file, output_file)


In [6]:
import random

# Định nghĩa ngữ pháp bằng từ điển
grammar = {
    'S': [['NP', 'VP']],
    'NP': [['Det', 'N'], ['Det', 'N', 'PP']],
    'VP': [['V', 'NP'], ['VP', 'PP']],
    'PP': [['P', 'NP']],
    'Det': [['the'], ['a']],
    'N': [['dog'], ['cat'], ['treat']],
    'V': [['chased'], ['saw'], ['liked']],
    'P': [['in'], ['on'], ['by']],
}

# Hàm sinh câu từ ngữ pháp
def generate_sentence(symbol):
    if symbol not in grammar:
        return symbol  # Nếu là ký hiệu terminal, trả về ngay
    
    production = random.choice(grammar[symbol])  # Chọn một quy tắc sản xuất ngẫu nhiên
    result = []
    for part in production:
        result.append(generate_sentence(part))  # Đệ quy cho mỗi phần
    return ' '.join(result)  # Kết hợp các phần lại thành một câu

# Hàm phân tích cú pháp
def parse_sentence(sentence):
    words = sentence.split()
    return parse_helper(words, 'S')

def parse_helper(words, symbol):
    if not words:
        return False
    
    if symbol not in grammar:
        if words[0] == symbol:
            return words[1:]  # Trả về danh sách từ còn lại
        else:
            return False

    for production in grammar[symbol]:
        remaining_words = words
        for part in production:
            remaining_words = parse_helper(remaining_words, part)
            if remaining_words is False:
                break
        if remaining_words is not False:
            return remaining_words  # Trả về danh sách từ còn lại

    return False

# Ví dụ sử dụng
print("Sinh câu:")
for _ in range(5):
    print(generate_sentence('S'))

print("\nPhân tích cú pháp câu:")
sentence = "the dog chased a cat"
if parse_sentence(sentence):
    print(f"Câu '{sentence}' được phân tích cú pháp thành công.")
else:
    print(f"Câu '{sentence}' không hợp lệ.")


Sinh câu:
a dog on the treat saw a cat
a dog on the treat liked the dog by a cat on the dog by a cat in the cat in a treat on the cat
a cat in a cat in the cat liked a dog in the treat in the dog in the treat by the treat by a cat in the dog on a treat
the cat on a treat in a cat saw the treat by the treat by the dog by a dog in a dog in the dog by a treat on a cat on the treat by the cat in a cat on a cat by a dog in the cat on the cat
a cat in the cat in a cat on the treat chased the cat on a cat by the cat

Phân tích cú pháp câu:
Câu 'the dog chased a cat' không hợp lệ.
